In [1]:
### SETTINGS ###
project_dir = '/homes_unix/agillig/github_repos/ginna' #specify your path to the github repository

n_perm = 100 # number of permutations per batch; reduced for the sake of the example (publication: 10,000)
n_batches = 1 # number of batches. can be useful to split the work in a cluster environment

compute_all_rsns = False # if True, compute all RSNs; if False, compute only RSN01 as an example

In [2]:
import sys
sys.path.insert(1, project_dir + '/code')

import func_toolbox as ftools
from func_toolbox import fetch_neurosynth_data
from nilearn import image
import null_parcellations

import numpy as np
import os
import pandas as pd
from pathlib import Path


neurosynth_terms_file = project_dir + '/data/terms/BCS_3D.csv'
os.makedirs(Path(neurosynth_terms_file).parent, exist_ok=True)

# download the file from the Pacela et al. 2021 paper repo 
# https://github.com/vale-pak/BCS

if not os.path.exists(neurosynth_terms_file):
    fetch_neurosynth_data(f'{project_dir}/data')

# https://github.com/vale-pak/BCS/blob/main/BCS_3D.csv
df = pd.read_csv(neurosynth_terms_file, sep = ',')

neurosynth_terms = df['Functions']

fcu = ftools.Utilities()
atlas_str = "aicha-aal3"

fcu.set_project_dir(project_dir)
fcu.set_atlas_name(atlas_str)

# create a new directory for the analysis
analysis_dir = project_dir + f'/analysis/mean_RSNs_{atlas_str}'
os.makedirs(analysis_dir, exist_ok=True)

mip_rsn_dir = project_dir + f'/Results/mip/mip_rsn_parcellated/aicha'


# Data processing

## create atlas : aicha + AAL cerebellum / BG

In [3]:
results = {}

In [4]:
parcellation_atlas_file = f'{project_dir}/data/parcellation_atlases/aicha-aal3/parcels_aicha-aal3.nii.gz'

# the atlas is already present in the github repository
if not os.path.exists(parcellation_atlas_file):
        atlases_dir = ''
        file_cortical = atlases_dir + '/AICHA_v2_websiteC/AICHA.nii'
        file_subcortical = atlases_dir + '/AAL3/AAL3v1.nii'

        cortical_img = image.load_img(file_cortical)
        new_data = np.zeros_like(cortical_img.get_fdata(), dtype = 'int32')

        cortical_data = image.load_img(file_cortical).get_fdata()
        subcortical_data = image.load_img(file_subcortical).get_fdata()
        # cortical
        max_cortical = np.max(cortical_data)
        for i, value in enumerate(np.unique(cortical_data)[1:]):
                new_data[cortical_data == value] = i + 1 
        # cerebellum
        # indices : from 95-120 for cerebellum; 121-170 for subcrotcial/brainstem
        for i, value in enumerate([l for l in range(95 , 121)]):
                new_data[subcortical_data == value] = max_cortical + i + 1 

        new_img = image.new_img_like(cortical_img, new_data)
        out_map = parcellation_atlas_file
        os.makedirs(os.path.dirname(out_map), exist_ok=True)
        new_img.to_filename(out_map)

In [5]:
#path to GINNA zstat maps
atlas_dir = f'{project_dir}/atlas/zmaps'
rsn_files = [os.path.join(atlas_dir, f) for f in os.listdir(atlas_dir) if f.endswith('.nii')]
rsn_files.sort()

In [6]:
# # # Create A 4D volume with all 506 maps of the model
# this may take a few min
terms_maps_dir = f'{project_dir}/data/dataset'
terms_maps_files = [os.path.join(terms_maps_dir, f) for f in os.listdir(terms_maps_dir) if f.endswith('.nii.gz')]
terms_maps_files.sort()

out_dir = f'{terms_maps_dir}/concatenated'
os.makedirs(out_dir, exist_ok=True)

out_name = out_dir + '/dataset_concatenated.nii.gz'
if os.path.isfile(out_name) == False:
    image.concat_imgs(terms_maps_files).to_filename(out_name)
concat_ds = image.load_img(out_name)

terms_maps_files_all = terms_maps_files

In [7]:
# Create & retrieve parcellated 506 meta analytic maps
# should also take a few minutes, but needs to be done only once

n_terms = concat_ds.shape[3]

parcellated_dataset_dir = f'{project_dir}/data/dataset/parcellated/{atlas_str}'
os.makedirs(parcellated_dataset_dir, exist_ok=True)

parcellated_dataset_file = os.path.join(parcellated_dataset_dir, f'neurosynthterms_parcellations_{atlas_str}.csv')

parcellated = []

if os.path.isfile(parcellated_dataset_file) == False:
    for t in range(n_terms):
        temp_img = image.index_img(concat_ds, t)
        parcellated.append(fcu.parcellate(temp_img, atlas=parcellation_atlas_file))

    parcellated = np.array(parcellated).squeeze()
    terms = neurosynth_terms.values
    # print(parcellated.shape)
    parcels = [i for i in range(1, parcellated.shape[1] +1)]
    dataset_parcellated = pd.DataFrame(parcellated, index=terms, columns = parcels)
    dataset_parcellated.to_csv(parcellated_dataset_file)
else:
    dataset_parcellated = pd.read_csv(parcellated_dataset_file, sep = ',', index_col = 0)

fcu.dataset_parcellated = dataset_parcellated


## Spatial correlation between RSNs & the Neurosynth maps

In [8]:
# for each rsn, compute the spatial correlation with the 506 meta analytic maps (parcellated)
atlas_img = '/homes_unix/agillig/Atlases/RSN_N41_zNpair_clean1.nii'
parcellation_dir = analysis_dir + '/parcellations'
os.makedirs(parcellation_dir, exist_ok=True)


rsn_str = pd.DataFrame(rsn_files).iloc[:,0].str.split('/').str[-1].str[-6:-4]

if compute_all_rsns == False:
    rsn_str = [rsn_str[0]] #limited to RSN01 to reduce computation time

for i, rsn in enumerate(rsn_str):
    parcellation_file = parcellation_dir + f'/rsn-{rsn}/rsn-{rsn}_unique_parcellated.csv'
    os.makedirs(os.path.dirname(parcellation_file), exist_ok=True)

    index = int(rsn) - 1
    # tmp_data = image.index_img(atlas_img, index)
    # affine = tmp_img.affine
    if os.path.isfile(parcellation_file) == False:
        tmp_img = image.load_img(rsn_files[index])
        tmp_parcellatd = fcu.parcellate(tmp_img, atlas=parcellation_atlas_file)
        np.savetxt(parcellation_file, tmp_parcellatd, delimiter = ',')
    data = np.loadtxt(parcellation_file, delimiter = ',')


    corr_temp = [np.corrcoef(data, dataset_parcellated.iloc[j,:])[1,0] for j in range(n_terms)]
    
    results[rsn] = {'term': neurosynth_terms, 'spatial_correlation': corr_temp}

# Non parametric statistics

In [9]:
# null_projections = {}
# use brainsmash to generate surrogate data that preserves spatial autocorrelation (Burt, 2020)
# https://github.com/murraylab/brainsmash: cf null_parcellations.py

# once computed, for each rsn, compute the correlation of the null parcellations with the 506 meta analytic maps (parcellated)

for rsn in rsn_str:
    null_parcellations.generate_null(1, project_dir, n_perm=n_perm, n_batches=n_batches)
    null_distr = fcu.compute_correlation_null(rsn='01', n_perm=n_perm, n_batches=n_batches, overwrite=True) #set overwrite to False if you have already computed the null distributions

    p, pcor = fcu.compute_pvalues(results[rsn]['spatial_correlation'], null_distr)

    results[rsn]['p'] = p
    results[rsn]['pcor'] = pcor
    results[rsn]['is_significant'] = (results[rsn]['pcor'] < 0.05).tolist()


processing rsn 01
batch 1
generating 100 surrogates
elapsed time: 26.73 s
saving surrogate maps to /homes_unix/agillig/github_repos/ginna/analysis/mean_RSNs_aicha-aal3/null_parcellations/rsn-01/rsn-01_null_parcellations_batch-01_of_1.csv
processing rsn 01
saving file


# Results summary for RSN01

In [10]:
res_table = pd.DataFrame.from_dict(results['01']).sort_values(by='spatial_correlation', ascending=False)
res_table

,term,spatial_correlation,p,pcor,is_significant
255,mnemonic,0.399498,0.009901,0.099010,False
362,retrieval,0.376930,0.009901,0.148515,False
349,recollection,0.371836,0.009901,0.158416,False
457,thoughts,0.318702,0.019802,0.336634,False
249,memory_retrieval,0.317122,0.009901,0.336634,False
...,...,...,...,...,...
272,movement,-0.105119,0.653465,1.000000,False
390,sensorimotor,-0.105715,0.683168,1.000000,False
267,motor_imagery,-0.107935,0.683168,1.000000,False
273,movements,-0.117540,0.673267,1.000000,False
